# Add fits files with same name from n different folders 

In [1]:
import os

from IPython.html import widgets
from IPython.display import display

import matplotlib.pyplot as plt

import glob
import pyfits

/Users/j35/anaconda/envs/py35/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
%gui qt

In [3]:
root_folder =  "/Volumes/My Book Thunderbolt Duo/IPTS/BraggEdge/SNAP/April9_2015/Overnight_Scan_DOE_Si_powder/"

In [4]:
try:
    from PyQt4 import QtCore, QtGui
    from PyQt4.QtGui import QFileDialog
except:
    from PyQt5 import QtCore, QtGui
    from PyQt5.QtWidgets import QFileDialog
    
def gui_fname(dir=None, index=0):
    """Select a file via a dialog and returns the file name.
    """
    if dir is None: dir ='./'
    message = "Select Folder #%d" %index
    dir_name = QFileDialog.getExistingDirectory(None, message,
                                                  dir,
                                                  QFileDialog.ShowDirsOnly)
    return dir_name

### Select folder n

#### Select all the folders to add... click CANCEL to let the program know you do not have any more folder to add 

In [9]:
def get_parent_folder(folder):
    return os.path.dirname(folder)

In [10]:
list_folders = []
%gui qt
while(True):
    folder_n = gui_fname(dir = root_folder)
    if folder_n is '':
        break
    if list_folders == []:
        root_folder = get_parent_folder(folder_n)
    list_folders.append(folder_n)

%matplotlib inline

### Check that all folders have the same number of images

In [11]:
number_of_files = []
list_files_in_folders = []
for _folder in list_folders:
    list_fname = glob.glob(_folder + '/*.fits')
    number_of_files.append(len(list_fname))
    list_files_in_folders.append(list_fname)

assert len(set(number_of_files)) == 1    

## Loading the data 

In [12]:
w = widgets.IntProgress()
w.max = len(list_folders)-1
display(w)

index = 0
data_folders = []

for index_folder, list_folder in enumerate(list_files_in_folders):

    _data_folder = []
    for index_file, filename in enumerate(list_folder):
        hdu_list = pyfits.open(filename)
        hdu = hdu_list[0]
        _data_folder.append(hdu.data)
    
    data_folders.append(_data_folder)
    w.value = index_folder

KeyboardInterrupt: 

In [8]:
import numpy as np
np.shape(data_folders)

(4, 1485, 512, 512)

## Retrieving images index

In [9]:
def digit_file_name(full_file_name):
    base_name = os.path.basename(full_file_name)
    [short_name, ext] = base_name.split('.')
    [image_name, image_sequence_number] = short_name.split('_')
    image_name_split = image_name.split('Image')
    image_number = image_name_split[1]
    return [image_name, image_number, image_sequence_number]

In [10]:
list_image_numbers = []
for index_folder, list_folder in enumerate(list_files_in_folders):
    _first_image = list_folder[0]
    [image_name, image_number, image_sequence_number] = digit_file_name(_first_image)
    list_image_numbers.append(image_number)
    
list_image_sequence = []
for index_file, _file in enumerate(list_files_in_folders[0]):
    [image_name, image_number, image_sequence_number] = digit_file_name(_file)
    list_image_sequence.append(image_sequence_number)
    

## Add data 

In [11]:
w = widgets.IntProgress()
number_of_files = len(data_folders[0])
w.max = number_of_files -1
display(w)

sum_data = []
sum_names = []
for index_file in range(number_of_files):

    _add_data = []
    tmp_name = ''
    for index_folder, folder_name in enumerate(list_files_in_folders):
        _list_data = data_folders[index_folder]
        _data = _list_data[index_file]
        [image_name, image_number, image_seqence_number] = digit_file_name(folder_name[index_file])
        if index_folder == 0:
            _add_data = _data
            _tmp_name = "%.2d" %int(image_number)
        else:
            _add_data += _data
            _tmp_name += '_%.2d' %int(image_number)
    
    sum_data.append(_add_data)
    sum_names.append(_tmp_name)
    w.value = index_file

## Exporting data 

In [12]:
def gui_fname(dir=None):
    """Select a folder
    """
    if dir is None: dir ='./'
    dir_name = QFileDialog.getExistingDirectory(None, "Select Folder ...",
                                                  dir,
                                                  QFileDialog.ShowDirsOnly)
    return dir_name

In [13]:
%gui qt
dir_name = gui_fname(dir = root_folder)
%matplotlib inline

In [14]:
w = widgets.IntProgress()
w.max = len(sum_data)-1
display(w)

for index in range(len(sum_data)):
    
    full_name = os.path.join(dir_name, sum_names[index] + '_' + list_image_sequence[index] + '.fits')

    hdu = pyfits.PrimaryHDU(sum_data[index])
    hdulist = pyfits.HDUList([hdu])
    hdulist.writeto(full_name)
    
    w.value = index